# 4章

In [1]:
from dotenv import load_dotenv
from pathlib import Path

env_path = Path(".") / ".env"
load_dotenv(dotenv_path=env_path)  # 環境変数を読み込む

True

## 4.2

In [4]:
from langchain_openai import OpenAI

model = OpenAI(model="gpt-3.5-turbo-instruct", temperature=0)
ai_message = model.invoke("こんにちは")
print(ai_message)



こんにちは

こんにちは、私はAIのアシスタントです。あなたのお手伝いをすることができます。何かお困りのことはありますか？


In [4]:
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

messages = [
    SystemMessage("You are a helpful assistant."),
    HumanMessage("こんにちは！私はジョンと言います"),
    AIMessage(content="こんにちは、ジョンさん！どのようにお手伝いできますか？"),
    HumanMessage(content="私の名前がわかりますか？"),
]

from pprint import pprint
pprint(messages)

[SystemMessage(content='You are a helpful assistant.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='こんにちは！私はジョンと言います', additional_kwargs={}, response_metadata={}),
 AIMessage(content='こんにちは、ジョンさん！どのようにお手伝いできますか？', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='私の名前がわかりますか？', additional_kwargs={}, response_metadata={})]


In [5]:
ai_message = model.invoke(messages)
print(ai_message.content)

はい、あなたの名前はジョンさんです。何か特別なことについてお話ししたいことがありますか？


In [7]:
messages = [
    SystemMessage("You are a helpful assistant."),
    HumanMessage("こんにちは！"),
]

for chunk in model.stream(messages):
    print(chunk.content, end="", flush=True)


こんにちは！どのようにお手伝いできますか？

### ローカルのGemmaでも動かしてみる

In [2]:
from langchain_huggingface import HuggingFacePipeline, ChatHuggingFace
from transformers import pipeline
import torch

# 1. Transformers 側でモデル＆トークナイザーを用意（ローカル読み込み）
# 例：Gemma のモデルをローカルから読み込む（量子化モデル等を使う想定）
text_gen = pipeline(
    "text-generation",
    model="google/gemma-3-1b-it",  # 小さめモデルを例に
    device_map="auto",
    torch_dtype="auto",
    # もし量子化モデルなら bitsandbytes 設定などを追加
)

# 2. LangChain の HuggingFacePipeline をラップ
hf_pipeline = HuggingFacePipeline(pipeline=text_gen)

# 3. ChatHuggingFace を使ってチャット形式で呼び出し
chat = ChatHuggingFace(llm=hf_pipeline)


/home/njz/tutorial_langchain/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
`torch_dtype` is deprecated! Use `dtype` instead!
Device set to use cuda:0


In [9]:
# 対話用メッセージリストを定義（System, Human, AI）
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage

messages = [
    SystemMessage(content="You are a helpful assistant."),
    HumanMessage(content="こんにちは！私はジョンと言います"),
    AIMessage(content="こんにちは、ジョンさん！どのようにお手伝いできますか？"),
    HumanMessage(content="私の名前がわかりますか？"),
]

# 応答取得
response = chat.invoke(messages)
print(response.content)

<bos><start_of_turn>user
You are a helpful assistant.

こんにちは！私はジョンと言います<end_of_turn>
<start_of_turn>model
こんにちは、ジョンさん！どのようにお手伝いできますか？<end_of_turn>
<start_of_turn>user
私の名前がわかりますか？<end_of_turn>
<start_of_turn>model
はい、ジョンさん、あなたの名前はジョンです。😊 どのようなお手伝いが必要ですか？


## 4.3 Prompt template

In [3]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template("""以下の料理のレシピを考えてください。

料理名: {dish}""")

prompt_value = prompt.invoke({"dish": "カレー"})
print(prompt_value.text)


以下の料理のレシピを考えてください。

料理名: カレー


In [13]:
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant."),
        MessagesPlaceholder("chat_history", optional=True),
        ("human", "{input}"),
    ]
)

prompt_value = prompt.invoke(
    {
        "chat_history": [
            HumanMessage(content="こんにちは！私はジョンと言います！"),
            AIMessage("こんにちは、ジョンさん！どのようにお手伝いできますか？"),
        ],
        "input": "私の名前が分かりますか？",
    }
)
pprint(dict(prompt_value))


{'messages': [SystemMessage(content='You are a helpful assistant.', additional_kwargs={}, response_metadata={}),
              HumanMessage(content='こんにちは！私はジョンと言います！', additional_kwargs={}, response_metadata={}),
              AIMessage(content='こんにちは、ジョンさん！どのようにお手伝いできますか？', additional_kwargs={}, response_metadata={}),
              HumanMessage(content='私の名前が分かりますか？', additional_kwargs={}, response_metadata={})]}


In [ ]:
from langsmith import Client

client = Client()

# ハブからプロンプトを取得
#   https://smith.langchain.com/hub/oshima/recipe?organizationId=ea7088b0-34ee-40cf-9303-d0aa6cd16441
prompt = client.pull_prompt("oshima/recipe")

prompt_value = prompt.invoke({"dish": "カレー"})
pprint(prompt_value.messages)


[SystemMessage(content='ユーザーが入力した料理のレシピを考えてください。', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='カレー', additional_kwargs={}, response_metadata={})]


In [17]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "user",
            [
                {"type": "text", "text": "画像を説明してください。"},
                {"type": "image_url", "image_url": {"url": "{image_url}"}},
            ],
        ),
    ]
)
image_url = "https://raw.githubusercontent.com/yoshidashingo/langchain-book/main/assets/cover.jpg"

prompt_value = prompt.invoke({"image_url": image_url})


In [18]:
model = ChatOpenAI(model="gpt-4o", temperature=0)
ai_message = model.invoke(prompt_value)
print(ai_message.content)


この画像は、「ChatGPT/LangChainによるチャットシステム構築[実践]入門」という本の表紙です。表紙にはカラフルな鳥のイラストが描かれています。著者は吉田真吾と大嶋勇樹です。表紙には、OpenAI APIやLangChainの活用方法についての内容が含まれていることが示されています。


## 4.4

In [26]:
from pydantic import BaseModel, Field


class Recipe(BaseModel):
    ingredients: list[str] = Field(description="ingredients of the dish")
    steps: list[str] = Field(description="steps to make the dish")


pprint(Recipe.model_json_schema())

{'properties': {'ingredients': {'description': 'ingredients of the dish',
                                'items': {'type': 'string'},
                                'title': 'Ingredients',
                                'type': 'array'},
                'steps': {'description': 'steps to make the dish',
                          'items': {'type': 'string'},
                          'title': 'Steps',
                          'type': 'array'}},
 'required': ['ingredients', 'steps'],
 'title': 'Recipe',
 'type': 'object'}


In [21]:
from langchain_core.output_parsers import PydanticOutputParser

output_parser = PydanticOutputParser(pydantic_object=Recipe)

In [27]:
format_instructions = output_parser.get_format_instructions()
print(format_instructions)


The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"ingredients": {"description": "ingredients of the dish", "items": {"type": "string"}, "title": "Ingredients", "type": "array"}, "steps": {"description": "steps to make the dish", "items": {"type": "string"}, "title": "Steps", "type": "array"}}, "required": ["ingredients", "steps"]}
```


In [28]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "ユーザーが入力した料理のレシピを考えてください。\n\n{format_instructions}",
        ),
        ("human", "{dish}"),
    ]
)

prompt_with_format_instructions = prompt.partial(
    format_instructions=format_instructions
)

In [29]:
prompt_value = prompt_with_format_instructions.invoke({"dish": "カレー"})
print("=== role: system ===")
print(prompt_value.messages[0].content)
print("=== role: user ===")
print(prompt_value.messages[1].content)


=== role: system ===
ユーザーが入力した料理のレシピを考えてください。

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"ingredients": {"description": "ingredients of the dish", "items": {"type": "string"}, "title": "Ingredients", "type": "array"}, "steps": {"description": "steps to make the dish", "items": {"type": "string"}, "title": "Steps", "type": "array"}}, "required": ["ingredients", "steps"]}
```
=== role: user ===
カレー


In [30]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

ai_message = model.invoke(prompt_value)
print(ai_message.content)

{
  "ingredients": [
    "鶏肉 500g",
    "玉ねぎ 2個",
    "にんじん 1本",
    "じゃがいも 2個",
    "カレールー 1箱",
    "水 800ml",
    "サラダ油 大さじ2",
    "塩 適量",
    "こしょう 適量"
  ],
  "steps": [
    "鶏肉は一口大に切り、塩とこしょうをふる。",
    "玉ねぎは薄切り、にんじんとじゃがいもは一口大に切る。",
    "鍋にサラダ油を熱し、玉ねぎを炒めて透明になるまで炒める。",
    "鶏肉を加え、表面が白くなるまで炒める。",
    "にんじんとじゃがいもを加え、さらに炒める。",
    "水を加え、沸騰したらアクを取り、弱火で20分煮る。",
    "カレールーを加え、よく溶かしてさらに10分煮る。",
    "味を見て、必要に応じて塩で調整する。",
    "ご飯と一緒に盛り付けて完成。"
  ]
}


In [32]:
recipe = output_parser.invoke(ai_message)
print(type(recipe))
pprint(recipe)

<class '__main__.Recipe'>
Recipe(ingredients=['鶏肉 500g', '玉ねぎ 2個', 'にんじん 1本', 'じゃがいも 2個', 'カレールー 1箱', '水 800ml', 'サラダ油 大さじ2', '塩 適量', 'こしょう 適量'], steps=['鶏肉は一口大に切り、塩とこしょうをふる。', '玉ねぎは薄切り、にんじんとじゃがいもは一口大に切る。', '鍋にサラダ油を熱し、玉ねぎを炒めて透明になるまで炒める。', '鶏肉を加え、表面が白くなるまで炒める。', 'にんじんとじゃがいもを加え、さらに炒める。', '水を加え、沸騰したらアクを取り、弱火で20分煮る。', 'カレールーを加え、よく溶かしてさらに10分煮る。', '味を見て、必要に応じて塩で調整する。', 'ご飯と一緒に盛り付けて完成。'])


In [33]:
from langchain_core.messages import AIMessage
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

ai_message = AIMessage(content="こんにちは。私はAIアシスタントです。")
ai_message = output_parser.invoke(ai_message)
print(type(ai_message))
print(ai_message)


<class 'str'>
こんにちは。私はAIアシスタントです。


## 4.5

In [34]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "ユーザーが入力した料理のレシピを考えてください。"),
        ("human", "{dish}"),
    ]
)

model = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

In [35]:
# Chain!
chain = prompt | model

In [36]:
ai_message = chain.invoke({"dish": "カレー"})
print(ai_message.content)

カレーのレシピをご紹介します！こちらは基本的なチキンカレーのレシピです。

### 材料（4人分）
- 鶏もも肉：400g
- 玉ねぎ：2個
- にんじん：1本
- じゃがいも：2個
- カレールー：1箱（約200g）
- サラダ油：大さじ2
- 水：600ml
- 塩：適量
- 胡椒：適量
- お好みでガーリックパウダーや生姜：少々

### 作り方
1. **下ごしらえ**：
   - 鶏もも肉は一口大に切り、塩と胡椒をふって下味をつけます。
   - 玉ねぎは薄切り、にんじんは輪切り、じゃがいもは一口大に切ります。

2. **炒める**：
   - 大きめの鍋にサラダ油を熱し、玉ねぎを中火で炒めます。玉ねぎが透明になるまで炒めます。
   - 鶏肉を加え、表面が白くなるまで炒めます。

3. **野菜を加える**：
   - にんじんとじゃがいもを鍋に加え、全体をよく混ぜます。

4. **煮る**：
   - 水を加え、沸騰したらアクを取り除きます。蓋をして中火で約15分煮ます。

5. **カレールーを加える**：
   - 火を弱め、カレールーを割り入れ、よく溶かします。さらに10分ほど煮込み、全体がなじんだら味を見て、必要に応じて塩や胡椒で調整します。

6. **仕上げ**：
   - お好みでガーリックパウダーや生姜を加えて風味をアップさせます。全体がとろっとしたら火を止めます。

7. **盛り付け**：
   - ご飯と一緒に盛り付けて、お好みで福神漬けやらっきょうを添えて完成です！

### おすすめのトッピング
- 煮卵
- チーズ
- パクチー

このレシピを参考に、ぜひ美味しいカレーを作ってみてください！お好みで具材を変えたり、辛さを調整したりして楽しんでくださいね。


In [37]:
print(type(chain))

<class 'langchain_core.runnables.base.RunnableSequence'>


In [38]:
from langchain_core.output_parsers import StrOutputParser

chain = prompt | model | StrOutputParser()
output = chain.invoke({"dish": "カレー"})
print(output)


カレーのレシピをご紹介します！こちらは基本的なチキンカレーのレシピです。

### 材料（4人分）
- 鶏もも肉：400g
- 玉ねぎ：2個
- にんじん：1本
- じゃがいも：2個
- カレールー：1箱（約200g）
- サラダ油：大さじ2
- 水：600ml
- 塩：適量
- 胡椒：適量
- お好みでガーリックパウダーや生姜：少々

### 作り方
1. **下ごしらえ**：
   - 鶏もも肉は一口大に切り、塩と胡椒をふって下味をつけます。
   - 玉ねぎは薄切り、にんじんは輪切り、じゃがいもは一口大に切ります。

2. **炒める**：
   - 大きめの鍋にサラダ油を熱し、玉ねぎを中火で炒めます。玉ねぎが透明になるまで炒めます。
   - 鶏肉を加え、表面が白くなるまで炒めます。

3. **野菜を加える**：
   - にんじんとじゃがいもを鍋に加え、全体をよく混ぜます。

4. **煮る**：
   - 水を加え、沸騰したらアクを取り除きます。蓋をして中火で約15分煮ます。

5. **カレールーを加える**：
   - 火を弱め、カレールーを割り入れ、よく溶かします。さらに10分ほど煮込み、全体がなじんだら味を見て、必要に応じて塩や胡椒で調整します。

6. **仕上げ**：
   - お好みでガーリックパウダーや生姜を加えて風味をアップさせます。全体がとろっとしたら火を止めます。

7. **盛り付け**：
   - ご飯と一緒に盛り付けて、お好みで福神漬けやらっきょうを添えて完成です！

### おすすめのトッピング
- チーズ
- 生卵（温泉卵や目玉焼き）
- パクチー

このレシピを参考に、ぜひ美味しいカレーを作ってみてください！お好みで具材を変えたり、スパイスを追加したりしてアレンジも楽しんでくださいね。


In [39]:
print(type(output))

<class 'str'>


In [42]:
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI


class Recipe(BaseModel):
    ingredients: list[str] = Field(description="ingredients of the dish")
    steps: list[str] = Field(description="steps to make the dish")


output_parser = PydanticOutputParser(pydantic_object=Recipe)


prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "ユーザーが入力した料理のレシピを考えてください。\n\n{format_instructions}",
        ),
        ("human", "{dish}"),
    ]
)

prompt_with_format_instructions = prompt.partial(
    format_instructions=output_parser.get_format_instructions()
)

model = ChatOpenAI(model="gpt-4o-mini", temperature=0).bind(
    response_format={"type": "json_object"}
)

In [43]:
# Chain!
chain = prompt_with_format_instructions | model | output_parser

In [44]:
recipe = chain.invoke({"dish": "カレー"})
print(type(recipe))
print(recipe)

<class '__main__.Recipe'>
ingredients=['鶏肉 500g', '玉ねぎ 2個', 'にんじん 1本', 'じゃがいも 2個', 'カレールー 1箱', '水 800ml', 'サラダ油 大さじ2', '塩 適量', 'こしょう 適量'] steps=['鶏肉は一口大に切り、塩とこしょうをふる。', '玉ねぎは薄切り、にんじんとじゃがいもは一口大に切る。', '鍋にサラダ油を熱し、玉ねぎを炒めて透明になるまで炒める。', '鶏肉を加え、表面が白くなるまで炒める。', 'にんじんとじゃがいもを加え、全体をよく混ぜる。', '水を加え、煮立ったらアクを取り除く。', '弱火にして、蓋をして約20分煮る。', 'カレールーを加え、よく溶かしてさらに10分煮込む。', '味を見て、必要に応じて塩で調整する。', 'ご飯と一緒に盛り付けて完成。']


## 4.6

In [47]:
from langchain_community.document_loaders import GitLoader


def file_filter(file_path: str) -> bool:
    return file_path.endswith(".md")


loader = GitLoader(
    clone_url="https://github.com/langchain-ai/langchain",
    repo_path="./langchain",
    branch="master",
    file_filter=file_filter,
)

In [48]:
raw_docs = loader.load()
print(len(raw_docs))


41


In [49]:
from langchain_text_splitters import CharacterTextSplitter

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)

docs = text_splitter.split_documents(raw_docs)
print(len(docs))

Created a chunk of size 1445, which is longer than the specified 1000
Created a chunk of size 1422, which is longer than the specified 1000
Created a chunk of size 1166, which is longer than the specified 1000


111


### 埋め込み

In [50]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")


In [ ]:
query = "AWSのS3からデータを読み込むためのDocument loaderはありますか？"

# クエリを埋め込みベクトル化
vector = embeddings.embed_query(query)
print(len(vector))
print(vector)


1536
[0.01968870498239994, -0.007562828715890646, 0.029617559164762497, -0.026554401963949203, 0.049940019845962524, 0.025012260302901268, -0.014977781102061272, 0.02129422128200531, 0.025730518624186516, -0.021463222801685333, 0.009273971430957317, -0.010615423321723938, -0.017417743802070618, -0.006010124925523996, -0.011428743600845337, 0.06316440552473068, 0.033335596323013306, -0.00034394499380141497, -0.04495446756482124, 0.02638540044426918, 0.0320047065615654, 0.03532136604189873, -0.03819439932703972, 0.021864602342247963, 0.01868525706231594, -0.018769757822155952, -0.020269649103283882, 0.032237086445093155, -0.00830221176147461, -0.10114755481481552, -0.009332065470516682, -0.0574183464050293, -0.034243982285261154, 0.04677123576402664, -0.023934874683618546, 0.034835487604141235, 0.023343367502093315, 0.013308888301253319, -0.00799589604139328, -0.033758100122213364, 0.00023254245752468705, -0.021009031683206558, 0.021072406321763992, 0.013171575032174587, 0.00648016110062

In [ ]:
from langchain_chroma import Chroma

# ドキュメントを埋め込みベクトル化してChromaに格納
db = Chroma.from_documents(docs, embeddings)

In [53]:
# 検索用のリトリーバーを作成
retriever = db.as_retriever()

In [55]:
query = "AWSのS3からデータを読み込むためのDocument loaderはありますか？"

context_docs = retriever.invoke(query)
print(f"len = {len(context_docs)}")

first_doc = context_docs[0]
print(f"metadata = {first_doc.metadata}")

len = 4
metadata = {'file_path': 'libs/README.md', 'file_name': 'README.md', 'source': 'libs/README.md', 'file_type': '.md'}


In [56]:
print(first_doc.page_content)

* [OpenAI](https://pypi.org/project/langchain-openai/)
* [Anthropic](https://pypi.org/project/langchain-anthropic/)
* [Ollama](https://pypi.org/project/langchain-ollama/)
* [DeepSeek](https://pypi.org/project/langchain-deepseek/)
* [xAI](https://pypi.org/project/langchain-xai/)
* and more

Most integrations have been moved to their own repositories for improved versioning, dependency management, collaboration, and testing. This includes packages from popular providers such as [Google](https://github.com/langchain-ai/langchain-google) and [AWS](https://github.com/langchain-ai/langchain-aws). Many third-party providers maintain their own LangChain integration packages.

For a full list of all LangChain integrations, please refer to the [LangChain Integrations documentation](https://docs.langchain.com/oss/python/integrations/providers).


In [57]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_template('''\
以下の文脈だけを踏まえて質問に回答してください。

文脈: """
{context}
"""

質問: {question}
''')

model = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)


In [58]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

output = chain.invoke(query)
print(output)


文脈にはAWSのS3からデータを読み込むためのDocument loaderに関する具体的な情報は含まれていません。ただし、AWSに関連するLangChainの統合があることが示されています。詳細な情報や具体的なDocument loaderについては、LangChainの統合ドキュメントを参照することをお勧めします。
